# Notebook pro trénink s destilací nad datasetem CIFAR10
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR10, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

In [1]:
%pip install transformers[torch] huggingface_hub datasets evaluate torchvision

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import knihoven a definice metod

In [2]:
from transformers import Trainer, TrainingArguments, MobileNetV2Config, MobileNetV2ForImageClassification, AutoModelForImageClassification
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn.functional as F
from PIL import Image
import torch.nn as nn
import numpy as np
import evaluate
import random
import pickle
import torch
import os

Resetování náhodného seedu pro replikovatelnost výsledků.
Zřejmě je možné části odebrat.

TODO: Odebrat zbytečná nastavení.

In [3]:
def reset_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

Nový wrapper, který pracuje přímo se soubory staženého a upraveného datasetu CIFAR10.
Využití načtení pomocí metody jako dříve není možné kvůli jiné checksum. 

Zároveň se již dotahují logity přímo z datasetu.

In [4]:
class CustomCIFAR10(Dataset):
    def __init__(self, root, train=True, transform=None, target_transform=None):
        self.root = root
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

        self.data = []
        self.targets = []
        self.logits = []
        
        if self.train:
             for i in range(1, 6):
                 data_file = os.path.join(self.root, 'cifar-10-batches-py', f'data_batch_{i}')
                 with open(data_file, 'rb') as fo:
                     dict = pickle.load(fo, encoding='bytes')
                     self.data.append(dict[b'data'])
                     self.targets.extend(dict[b'labels'])
                     self.logits.extend(dict[b'logits'])  
        else:
            data_file = os.path.join(self.root, 'cifar-10-batches-py', 'test_batch')
            with open(data_file, 'rb') as fo:
                dict = pickle.load(fo, encoding='bytes')
                self.data.append(dict[b'data'])
                self.targets.extend(dict[b'labels'])
                self.logits.extend(dict[b'logits'])  

        self.data = np.concatenate(self.data, axis=0)
        self.targets = np.array(self.targets)
        self.logits = np.array(self.logits)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index].reshape(3, 32, 32).transpose(1, 2, 0)
        label = self.targets[index]
        logit = self.logits[index]
        
        image = Image.fromarray(image.astype('uint8'), 'RGB')
        logit = torch.tensor(logit, dtype=torch.float)
        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            target = self.target_transform(target)
            
        return {
            'pixel_values': image,
            'labels': label,
            'logits': logit
        }


Definice accuracy metriky pro trénování modelu.

In [5]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    acc = accuracy.compute(references=labels, predictions=np.argmax(predictions, axis=1))
    return {"accuracy": acc["accuracy"]}

Trénovací argumenty pro trainer. 

In [6]:
def get_training_args(output_dir, logging_dir, remove_unused_columns):
    return (
        TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5, #Defaultní hodnota 
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        num_train_epochs=20,
        weight_decay=0.01,
        seed = 42,  #Defaultní hodnota 
        metric_for_best_model="accuracy",
        load_best_model_at_end=True,
        fp16=True, 
        logging_dir=logging_dir,
        remove_unused_columns=remove_unused_columns,
    ))


Náhodně inicializovaný MobileNetV2.

In [7]:
def get_random_init_mobilenet():
    reset_seed(42)
    student_config = MobileNetV2Config()
    student_config.num_labels = 10
    return MobileNetV2ForImageClassification(student_config)

Zamražení modelu a trénink pouze klasifikační hlavy.

In [8]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False

    for param in model.classifier.parameters():
        param.requires_grad = True

Inicializovaný MobileNetV2.

In [9]:
def get_mobilenet():
    model_pretrained = MobileNetV2ForImageClassification.from_pretrained("google/mobilenet_v2_1.0_224")
    in_features = model_pretrained.classifier.in_features

    model_pretrained.classifier = nn.Linear(in_features,10) #Úprava klasifikační hlavy
    model_pretrained.num_labels = 10
    model_pretrained.config.num_labels = 10

    return model_pretrained

In [10]:
reset_seed(42)

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Provedení transformací nad datasetem.

In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
test = CustomCIFAR10(root='../data/10-logits', train=False, transform=transform)
train = CustomCIFAR10(root='../data/10-logits', train=True, transform=transform)

### Standardní trénink náhodně inicializovaného modelu. 

In [13]:
training_args = get_training_args("./results/cifar10-random", './logs/cifar10-random', True)
model = get_random_init_mobilenet()

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.999400,1.465344,0.461900
2,1.338000,1.238446,0.563300
3,1.135700,1.040728,0.626200
4,0.888400,0.949196,0.677800
5,0.773600,0.940272,0.678500
6,0.618400,0.835595,0.726800
7,0.558500,1.199692,0.655300
8,0.431500,1.572898,0.602600
9,0.356000,0.850076,0.743500
10,0.288900,0.897842,0.753100


TrainOutput(global_step=15640, training_loss=0.45282880482466326, metrics={'train_runtime': 5340.8309, 'train_samples_per_second': 187.237, 'train_steps_per_second': 2.928, 'total_flos': 2.020099608576e+18, 'train_loss': 0.45282880482466326, 'epoch': 20.0})

In [16]:
model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [17]:
trainer.evaluate()

{'eval_loss': 0.8750341534614563,
 'eval_accuracy': 0.756,
 'eval_runtime': 26.6653,
 'eval_samples_per_second': 375.019,
 'eval_steps_per_second': 5.888,
 'epoch': 20.0}

## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

In [18]:
class ImageDistilTrainer(Trainer):
    def __init__(self, student_model=None, temperature=None, lambda_param=None, *args, **kwargs):
        super().__init__(model=student_model, *args, **kwargs)
        self.student = student_model
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.temperature = temperature
        self.lambda_param = lambda_param



    def compute_loss(self, student, inputs, return_outputs=False, num_items_in_batch=None):
        logits = inputs.pop("logits")

        student_output = self.student(**inputs)

        soft_teacher = F.softmax(logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)


        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)


        student_target_loss = student_output.loss

        loss = ((1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss)
        return (loss, student_output) if return_outputs else loss

### Trénink náhodně inicializovaného modelu s pomocí destilace znalostí

In [19]:
reset_seed(42)

In [20]:
student_model = get_random_init_mobilenet()

In [21]:
training_args = get_training_args("./results/cifar10-random-KD", './logs/cifar10-random-KD', False)

In [22]:
trainer = ImageDistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    temperature = 5,
    lambda_param = 0.3
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.701900,1.622049,0.419600
2,1.225600,1.474280,0.533700
3,1.069000,1.287490,0.639800
4,0.856300,1.303758,0.673000
5,0.766500,1.283496,0.669200
6,0.644600,1.225594,0.704700
7,0.599200,1.341409,0.683400
8,0.502900,1.443276,0.617300
9,0.444300,1.215594,0.723000
10,0.395300,1.173633,0.762400


TrainOutput(global_step=15640, training_loss=0.5219732990655143, metrics={'train_runtime': 5947.577, 'train_samples_per_second': 168.136, 'train_steps_per_second': 2.63, 'total_flos': 2.020099608576e+18, 'train_loss': 0.5219732990655143, 'epoch': 20.0})

In [24]:
student_model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [25]:
trainer.evaluate()

{'eval_loss': 1.1354732513427734,
 'eval_accuracy': 0.7626,
 'eval_runtime': 27.6776,
 'eval_samples_per_second': 361.302,
 'eval_steps_per_second': 5.672,
 'epoch': 20.0}

## Získání inicializovaného MobileNetV2 modelu

In [26]:
reset_seed(42)

In [27]:
model_pretrained = get_mobilenet()

In [28]:
print(model_pretrained)

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [29]:
freeze_model(model_pretrained)

In [30]:
training_args = get_training_args("./results/cifar10-pretrained-head", './logs/cifar10-pretrained-head', True)

In [31]:
trainer = Trainer(
    model=model_pretrained,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.890400,1.359444,0.630800
2,1.122600,1.207275,0.624100
3,1.016800,0.971210,0.709600
4,0.904100,0.907923,0.714900
5,0.876800,0.984846,0.680700
6,0.832400,0.870077,0.721900
7,0.827400,1.021028,0.655200
8,0.802100,1.105069,0.629400
9,0.791900,0.866780,0.706900
10,0.785000,0.874822,0.712600


TrainOutput(global_step=15640, training_loss=0.8626149301943572, metrics={'train_runtime': 3635.8498, 'train_samples_per_second': 275.039, 'train_steps_per_second': 4.302, 'total_flos': 2.020099608576e+18, 'train_loss': 0.8626149301943572, 'epoch': 20.0})

In [33]:
model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [34]:
trainer.evaluate()

{'eval_loss': 0.7993389368057251,
 'eval_accuracy': 0.7356,
 'eval_runtime': 27.842,
 'eval_samples_per_second': 359.17,
 'eval_steps_per_second': 5.639,
 'epoch': 20.0}

### Trénink inicializovaného MobileNetV2

In [35]:
reset_seed(42)

In [36]:
model_pretrained_whole = get_mobilenet()

In [37]:
training_args = get_training_args("./results/cifar10-pretrained", './logs/cifar10-pretrained', True)

In [38]:
trainer = Trainer(
    model=model_pretrained_whole,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
)

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.667800,0.352117,0.881500
2,0.182900,0.473985,0.855200
3,0.109900,0.307834,0.908600
4,0.051500,0.306266,0.913600
5,0.035600,0.562306,0.874200
6,0.016000,0.446769,0.906100
7,0.013500,0.618080,0.873600
8,0.008400,0.747778,0.854000
9,0.005900,0.461412,0.911700
10,0.003800,0.434470,0.919800


TrainOutput(global_step=15640, training_loss=0.048249347503666225, metrics={'train_runtime': 5404.6192, 'train_samples_per_second': 185.027, 'train_steps_per_second': 2.894, 'total_flos': 2.020099608576e+18, 'train_loss': 0.048249347503666225, 'epoch': 20.0})

In [40]:
model_pretrained_whole.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [41]:
trainer.evaluate()

{'eval_loss': 0.4153886139392853,
 'eval_accuracy': 0.9295,
 'eval_runtime': 30.5178,
 'eval_samples_per_second': 327.678,
 'eval_steps_per_second': 5.145,
 'epoch': 20.0}

## Trénink s pomocí destilace znalostí inicializovaného MobileNetV2

### Trénink inicializovaného modelu - pouze klasifikační hlavy s pomocí destilace

In [42]:
reset_seed(42)

In [43]:
student_model_pretrained = get_mobilenet()

In [44]:
freeze_model(student_model_pretrained)

In [45]:
training_args = get_training_args("./results/cifar10-pretrained-head-KD", './logs/cifar10-pretrained-head-KD', False)

In [46]:
trainer = ImageDistilTrainer(
    student_model=student_model_pretrained,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    temperature = 5,
    lambda_param = 0.6
)

In [47]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.194200,1.295173,0.638000
2,0.773000,1.352618,0.638800
3,0.734400,1.305183,0.712900
4,0.694700,1.334172,0.714900
5,0.687300,1.368952,0.684700
6,0.671700,1.329848,0.720300
7,0.670400,1.372895,0.664700
8,0.661400,1.381742,0.639500
9,0.658500,1.350707,0.703500
10,0.657100,1.364204,0.720200


TrainOutput(global_step=15640, training_loss=0.688703476498499, metrics={'train_runtime': 3721.5092, 'train_samples_per_second': 268.708, 'train_steps_per_second': 4.203, 'total_flos': 2.020099608576e+18, 'train_loss': 0.688703476498499, 'epoch': 20.0})

In [48]:
student_model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [49]:
trainer.evaluate()

{'eval_loss': 1.321333408355713,
 'eval_accuracy': 0.7356,
 'eval_runtime': 30.0257,
 'eval_samples_per_second': 333.048,
 'eval_steps_per_second': 5.229,
 'epoch': 20.0}

### Trénink inicializovaného modelu s pomocí destilace

In [50]:
reset_seed(42)

In [51]:
student_model_pretrained_whole = get_mobilenet()

In [52]:
training_args = get_training_args("./results/cifar10-pretrained-KD", './logs/cifar10-pretrained-KD', False)

In [53]:
trainer = ImageDistilTrainer(
    student_model=student_model_pretrained_whole.to(device),
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    temperature = 5,
    lambda_param = 0.4
)

In [54]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.588200,1.113030,0.886800
2,0.246500,1.138879,0.864400
3,0.196800,1.028809,0.921600
4,0.155900,0.991875,0.925200
5,0.139800,1.172694,0.873700
6,0.122300,1.053637,0.913400
7,0.118000,1.166352,0.882600
8,0.109900,1.100621,0.878000
9,0.105700,1.038172,0.919400
10,0.104200,0.995655,0.932400


TrainOutput(global_step=15640, training_loss=0.13773624153088426, metrics={'train_runtime': 5781.8021, 'train_samples_per_second': 172.956, 'train_steps_per_second': 2.705, 'total_flos': 2.020099608576e+18, 'train_loss': 0.13773624153088426, 'epoch': 20.0})

In [55]:
student_model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [56]:
trainer.evaluate()

{'eval_loss': 0.9895767569541931,
 'eval_accuracy': 0.936,
 'eval_runtime': 27.2427,
 'eval_samples_per_second': 367.071,
 'eval_steps_per_second': 5.763,
 'epoch': 20.0}